In [1]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium

In [17]:
#get mrt data
mrt_df = pd.read_csv('../data/mrt_station_final.csv')
mrt_df.head()
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()

# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load /Users/ethan/Documents/GitHub/Public-Transportation-In-Singapore/Backend/otp/  --serve

# Sample API request to the OTP isochrone endpoint
# response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()


Write a function for fetching isochrome. Configure params here

In [18]:

def fetch_isochrone(lat, lon, modes, cutoff="20M17S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-04-12T10:19:03+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None


Visualise all isochrones

In [81]:

m = folium.Map(location=coords[0], zoom_start=12)

# Fetch and add isochrones for each MRT station
for lat, lon in coords:
    isochrone_geojson = fetch_isochrone(lat, lon)
    if isochrone_geojson:
        folium.GeoJson(isochrone_geojson, name=f"Isochrone {lat}, {lon}").add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m


KeyboardInterrupt: 

In [ ]:
#1 cycling range
#2 walking range
#3 bus + walking range
#https://github.com/gis-ops/routingpy

Saving isochrones for bus, walking, cycling.

In [19]:
# Assuming mrt_df is already defined
mrt_df['bus 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BUS,WALK", cutoff="20M17S"), axis=1) #change modes to BUS/WALK
mrt_df['walking 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "WALK", cutoff="20M17S"), axis=1) #change modes to WALK
mrt_df['cycling 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BICYCLE", cutoff="20M17S"), axis=1) #change modes to CYCLE


save data as .pkl to preserve json

In [20]:
#mrt_df.to_pickle("../data/mrt_df_with_isochrones.pkl")
mrt_df = pd.read_pickle("../data/mrt_df_with_isochrones.pkl")

In [27]:
import folium

m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in mrt_df.iterrows():
    # Creating a popup message with a brief description or link
    popup_message = f"MRT Station: {index}<br>"
    popup_message += "Cycling 30M Isochrone available."  # Example message; customize as needed
    
    # Create and add a marker to the map
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_message, parse_html=True, max_width=450)
    ).add_to(m)
m




AttributeError: 'dict' object has no attribute 'str'